# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに関連して、強化学習を用いて「20の質問」ゲームをプレイするためのアルゴリズムを開発することを目的としています。著者はMarília Prata（mpwolke）で、2024年5月15日に公開されました。

### 問題の概要
「20の質問」ゲームにおいて、プレイヤーは特定のオブジェクトを当てるために質問を重ねていく必要があり、その質問の選択がゲームの勝敗に重要な役割を果たします。本ノートブックは、エージェントがユーザーとのインタラクションを通じて最適な質問アプローチを学ぶために強化学習（RL）を用いた手法に焦点を当てています。

### 使用されている手法とライブラリ
1. **データ処理**: pandasとnumpyを用いてデータ処理を行い、さまざまなJSONファイルからデータを読み込みます。
2. **強化学習**:
   - **政策ベースの強化学習**: 提案された手法では、エージェントはマルコフ決定過程（MDP）を通じて質問選択プロセスを最適化します。信頼度をモデル化し、ユーザーからの回答に基づいて信頼度を更新します。
   - **報酬ネットワーク**: 各タイムステップで適切な即時報酬を推定するために使用され、エージェントは誤った回答による局所的最適から脱出することが可能になります。
3. **Keras**: Kerasを使用して深層学習モデルを構築し、特にGemmaCausalLMモデルを利用して質問生成を行います。こちらはGoogleのGeminiモデルに基づいた大規模な言語モデルです。
4. **データの整形と正規化**: JSONデータをDataFrameに変換し、エラーが発生した場合のデータ整形や正規化を行っています。

このノートブックは、20の質問ゲームを効率的にプレイするための戦略的な質問選択を強化学習によって実現するための実験的なアプローチを提供しています。最終的には、特定のオブジェクトに関する知識を持たない状態で、どうにかしてゲームを進めるための手法を模索している点が特徴です。

---


# 用語概説 
以下は、Jupyter Notebookの内容を基に、機械学習・深層学習に関する初心者がつまずきそうなマイナーな専門用語の解説です。一部はこのノートブック特有のドメイン知識に関連した内容です。

---

1. **政策ベースの強化学習 (Policy-based Reinforcement Learning)**  
   エージェントが最適な行動を選ぶための政策（方針）を直接学習する手法。価値ベースの手法（Q学習など）とは異なり、行動の選択確率を直接最適化することで、連続的なアクション空間に対しても適用しやすいという利点があります。

2. **報酬ネットワーク (Reward Network)**  
   強化学習の文脈で、エージェントが状態に対して受け取る報酬を予測するためのモデル。このネットワークは、エージェントの行動に対するフィードバックを強化し、学習を効果的に支援します。

3. **エンジニアリングトリック (Engineering Tricks)**  
   機械学習モデルの性能を向上させるために用いる技術的手法。具体的な手法は多岐にわたり、データ処理や正則化技術、異常値処理などが含まれます。ドメイン固有の問題に対処するための工夫が必要です。

4. **マルコフ決定過程 (Markov Decision Process: MDP)**  
   エージェントの選択肢とそれに対する報酬と状態遷移が定義された数学的枠組み。各状態の選択によって次の状態が決まるため、過去の状態は現在の状態に影響を与えない（マルコフ性）ことが前提です。強化学習における基本的な理論です。

5. **信頼度 (Confidence)**  
   特定の状態においてエージェントがその行動が適切であると考える程度。質問の選択において、どれくらいの確信を持って次の行動を選んでいるかを示す指標。

6. **即時報酬 (Immediate Reward)**  
   エージェントがある行動を取った直後に受け取る報酬。強化学習においては、即時報酬を考慮に入れながら長期的な報酬を最大化することが目標になります。

7. **長期的リターン (Long-term Return)**  
   エージェントが今この瞬間の行動によって将来的に得られる報酬の合計値。これを最大化することが強化学習の目標であり、エージェントは即時報酬と長期的なリターンを考慮して政策を決定します。

8. **局所最適 (Local Optima)**  
   最適解ではないが、周辺の解よりも優れている状態。機械学習の最適化問題では、アルゴリズムがこの局所最適に陥ってしまうことがあります。これを避けるために、様々な戦略（例：エクスプロレーション）を考慮する必要があります。

9. **サンプリング質問 (Sampling Questions)**  
   エージェントが質問を選ぶ際に、単に最も効果的と思われる質問を選ぶのではなく、いくつかの選択肢からランダムに選ぶアプローチ。これによって多様性が生まれ、結果的にエージェントのパフォーマンス向上につながる可能性があります。

10. **ノイズのある回答 (Noisy Responses)**  
    ユーザーからの回答が必ずしも正確でない、あるいは曖昧なものであること。実際のアプリケーションにおいては常に存在する問題であり、モデルがこのようなノイズに対処できるように設計されることが求められます。

--- 

これらの用語の理解が、ノートブックや強化学習の文脈において重要な要素となります。

---


2024年5月15日公開。著者: Marília Prata, mpwolke


In [ ]:
# このPython 3環境には、便利な分析ライブラリが多数インストールされています
# これは、kaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数
import pandas as pd # データ処理, CSVファイルの入出力 (例: pd.read_csv)

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# 例えば、このコードを実行することで（実行をクリックするかShift+Enterを押すと）、入力ディレクトリ内のすべてのファイルをリスト表示します

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 現在のディレクトリ（/kaggle/working/）には最大20GBのデータを書き込むことができ、これは"Save & Run All"を使用してバージョンを作成する際に出力として保存されます
# また、一時ファイルは/kaggle/temp/に書き込むことができますが、現在のセッションの外では保存されません

# コンペティションの引用

@misc{llm-20-questions,
    author = {Zoe Mongan, Luke Sernau, Will Lifferth, Bovard Doerschuk-Tiberi, Ryan Holbrook, Will Cukierski, Addison Howard},
    title = {LLM 20 Questions},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/llm-20-questions}
}

# 鳥ですか？飛行機ですか？いいえ、カグラーです！

![](https://img1.picmix.com/output/pic/normal/5/6/8/3/11323865_2f559.gif)https://en.picmix.com/pic/bird-plane-bert-11323865

# 強化学習によるQ20ゲーム

政策ベースの強化学習を用いた20の質問ゲームのプレイ

著者: Huang Hu1, Xianchao Wu, Bingfeng Luo, Chongyang Tao, Can Xu, Wei Wu, Zhan Chen

「この論文では、著者らは質問者エージェントがユーザーとの継続的なインタラクションを通じて質問選択の最適政策を学ぶことを可能にする新しい政策ベースの強化学習（RL）手法を提案しました。トレーニングを促進するために、より情報価値の高い報酬を推定するために報酬ネットワークを使用することも提案されました。以前の手法と比較して、彼らのRL手法はノイズのある回答に対して強固であり、物体の知識ベースに依存しません。実験結果は、彼らのRL手法がエントロピーに基づくエンジニアリングシステムを明らかに上回り、ノイズのないシミュレーション環境で競争力のあるパフォーマンスを持つことを示しています。」

「Q20ゲームシステムを構築するためのアルゴリズムを設計することは簡単ではありません。決定木ベースの手法はQ20ゲームに自然に適しているように思われますが、通常は各オブジェクトに関する十分な情報を含む明確に定義された知識ベース（KB）が必要であり、これは実際には利用可能ではありません。質問と物体の選択のためのピボットとして、既存のKB（知識ベース）に依存しないオブジェクト質問関連テーブルが使用されました。さらに、この関連テーブルは、多くのエンジニアリングトリックを用いて改善されました。これらのテーブルベースの手法は質問を貪欲に選択し、モデルパラメータはルールによってのみ更新されるため、彼らのモデルはユーザーからのノイズのある回答に非常に敏感です。これはリアルワールドのQ20ゲームでは一般的です。一般化能力を向上させるために価値ベースの強化学習（RL）モデルが利用されましたが、依然として既存のKBに依存しています。」

「この論文では、ゲームの質問選択プロセスをマルコフ決定過程（MDP）として定式化し、Q20ゲームでの質問選択の最適政策を学ぶための新しい政策ベースのRLフレームワークを提案しました。彼らの質問者エージェントは、ターゲットオブジェクトに対する信頼度をモデル化するためにすべての物体に対する確率分布を維持し、ユーザーからの回答に基づいて信頼度を更新します。」

「各タイムステップで、エージェントは政策ネットワークを使用して信頼度ベクトルを入力し、次の質問を選択するための質問分布を出力します。選択された各質問に即時報酬がないという問題を解決するために、著者はリワードネットを導入して各タイムステップで適切な即時報酬を推定することを提案しました。これはさらに、RLモデルをトレーニングするための長期的なリターンを計算するために使用されます。」

「彼らのRL（強化学習）フレームワークは、モデルパラメータが完全に学習可能であり、質問分布が質問をサンプリングするための原則的な方法を提供するため、ノイズのある回答に対してエージェントを堅牢にします。これにより、エージェントは誤った回答による局所的最適から脱出し、トレーニング中にモデルの一般化能力を向上させるためのさらなるランダム性を導入します。さらに、サンプリング質問の能力は貪欲な選択と比較して、エージェントが尋ねる質問の多様性を改善します。これはユーザー体験にとって重要です。」

https://www.researchgate.net/publication/327199595_Playing_20_Question_Game_with_Policy-Based_Reinforcement_Learning

# llm_20_questionsのjsonファイル


In [ ]:
# df = pd.read_json(path_or_buf='/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.json')

# df= pd.read_json('../input/llm-20-questions/llm_20_questions/llm_20_questions.json', lines=True)
df= pd.read_json('../input/llm-20-questions/llm_20_questions/llm_20_questions.json', typ="series")
df.head()

# 矢印の近くには: エージェント、設定、報酬、観察、アクション、ステータスがあります

「Mixing dicts with non-Series may lead to ambiguous ordering」というエラーが発生しました

データ型が様々で、いくつかは文字列、いくつかはリスト、複数の{}などがあるため、このエラーはデータを正規化することで解決できるかもしれません。

https://www.kaggle.com/code/mpwolke/trafic-json-mixing-dicts-with-non-series


In [ ]:
# Mpwolke https://www.kaggle.com/code/mpwolke/trafic-json-mixing-dicts-with-non-series

# StackOverflow: https://stackoverflow.com/questions/49505872/read-json-to-pandas-dataframe-valueerror-mixing-dicts-with-non-series-may-lea

import json
import pandas as pd
ques = json.load(open('../input/llm-20-questions/llm_20_questions/llm_20_questions.json'))

df = pd.DataFrame(ques["observation"])

In [ ]:
df.head()

In [ ]:
df1 = pd.DataFrame(ques["configuration"])
df1.head()

# 設定の転置


In [ ]:
df1_transposed = df1.T
df1_transposed.head()

In [ ]:
df2 = pd.DataFrame(ques["status"])
df2.head()

# Kerasのインストール


In [ ]:
# Kerasを最後にインストールします。詳細はhttps://keras.io/getting_started/を参照してください。
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

import os

os.environ["KERAS_BACKEND"] = "jax"  # または "torch" または "tensorflow".
# JAXバックエンドでメモリの断片化を避けるため
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

import keras
import keras_nlp

# Gemma_2b_en

最初のノートブックはGemma 7bを適用することを予定していましたが、解決できない予期しないエラーが発生しました。そこで、Gemma 2bを使用することにしました。ただし、意図した質問のデータセットを持っていないので、言い換えれば、ジンクスがかかってしまいました。もう一つのジンクスがかかったコードのコレクションに追加されました。


In [ ]:
%%time

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

# すべての回答は「はい」であり、最後の1つを除いています。


In [ ]:
print(gemma_lm.generate("Is Gemma, an open model based on Google's Gemini models?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Is Gemma a lightweight, text-to-text, decoder-only large language model?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Is Gemma a support low-latency generative AI used in cases such as streaming text?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Has Gemma 27 billion parameters?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Is LLaVA-Gemma, a MMFM that leverages the powerful Gemma language models for efficient multimodal interactions?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Is JAX a framework developed by Google?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Are you a model Gemma?", max_length=1024))

In [ ]:
print(gemma_lm.generate("Are you participating on this Q20 game Gemma?", max_length=1024))

![](https://www.wallyandosborne.com/wp-content/uploads/2006/02/2005-11-01.gif)https://www.wallyandosborne.com/wp-content/uploads/2006/02/2005-11-01.gif

# 謝辞:

mpwolke https://www.kaggle.com/code/mpwolke/eureka-gemma-1-1-instruct-7b-en

https://www.kaggle.com/code/mpwolke/thanks-for-delivering-my-writeup-order-gemma/notebook

mpwolke https://www.kaggle.com/code/mpwolke/trafic-json-mixing-dicts-with-non-series


---

# コメント 

> ## SuM
> 
> 面白いゲームですね。共有してくれてありがとう😀
> 
> 
> 


---

> ## GODDiao
> 
> LOL、どうやってそれをエージェントにするのでしょうか？

> ## Marília Prata トピックの著者
> > それについてはまったく分かりません、GODDiao : )

---

> ## Ana Pedra López
> 
> 面白いタスクですね！

> ## Marília Prata トピックの著者
> > 確かに、Ana。ただファイルを開いて初めて、このタスクがどれほど難しいかを理解しました。私の知識はとても乏しいです。
> 

---

> ## Márcio Santos
> 
> ありがとう共有してくれて!!!

---

> ## Musa Khan
> 
> 簡単でユーモラスであることがすべてです！

---

> ## Marília Prata トピックの著者
> > あなたのサポートに感謝しています、Fatih。ありがとう。
> 
---